<a href="https://colab.research.google.com/github/EnaJeong/LogInAndOut-log-severity-level-analysis/blob/jihoon/EDA/EDA_jh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dataframe

In [ ]:
import os 
import pandas as pd

import re
import numpy as np
import seaborn as sbn

from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# 데이터 전체 출력을 보기 위함
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [ ]:
%%time
data_path = os.path.join('data')

train_df = pd.read_csv(os.path.join(data_path,'/content/drive/MyDrive/log_data/train.csv'))
validation_df = pd.read_csv(os.path.join(data_path,'/content/drive/MyDrive/log_data/validation_sample.csv'))
test_df = pd.read_csv(os.path.join(data_path,'/content/drive/MyDrive/log_data/test.csv'))
submission_df = pd.read_csv(os.path.join(data_path,'/content/drive/MyDrive/log_data/sample_submission.csv'))

CPU times: user 8.63 s, sys: 1.07 s, total: 9.7 s
Wall time: 18.6 s


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472972 entries, 0 to 472971
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        472972 non-null  int64 
 1   level     472972 non-null  int64 
 2   full_log  472972 non-null  object
dtypes: int64(2), object(1)
memory usage: 10.8+ MB


In [ ]:
train_df = train_df.astype(dtype={'full_log': 'string'})
print(train_df.dtypes)

id           int64
level        int64
full_log    string
dtype: object


In [ ]:
train_df.shape

(472972, 3)

In [ ]:
# 레벨별 데이터 확인 
train_df['level'].value_counts().sort_index()

0    334065
1    132517
2    12    
3    4141  
4    10    
5    2219  
6    8     
Name: level, dtype: int64

In [ ]:
# 등급 별 샘플 데이터프레임 추출
sample_df = pd.DataFrame(columns=['id', 'level', 'full_log'])
for i in train_df['level'].unique():
    temp_df = train_df.query('level==@i').sample()
    sample_df = pd.concat([sample_df, temp_df])
sample_df.sort_values(by=['level'], axis=0, inplace=True)

# 등급 별 full_log로 보기 
for i in range(len(sample_df)):
    print('level: ', i, '\n',sample_df.iloc[i, 2])
    print()

level:  0 
 Jan 25 22:19:06 localhost logstash: [2021-01-25T22:19:06,207][ERROR][logstash.inputs.jdbc     ] Unable to connect to database. Tried 1 times {:error_message=>"Java::ComMysqlJdbcExceptionsJdbc4::CommunicationsException: Communications link failure\n\nThe last packet sent successfully to the server was 0 milliseconds ago. The driver has not received any packets from the server."}

level:  1 
 Jan 23 08:00:02 localhost sudo: pam_unix(sudo:session): session closed for user root

level:  2 
 The average number of logs between 14:00 and 15:00 is 9734. We reached 26958.

level:  3 
 Nov 18 14:04:03 sv260 sshd[8646]: Failed password for root from 61.41.101.142 port 42390 ssh2

level:  4 
 Nov 8 20:25:42 sv260 sshd[37597]: Did not receive identification string from 192.168.0.181

level:  5 
 oscap: msg: "xccdf-result", scan-id: "0001605570454", content: "ssg-centos-7-ds.xml", title: "Set Password Minimum Length", id: "xccdf_org.ssgproject.content_rule_accounts_password_pam_minlen", 

In [ ]:
# 중복된 데이터 확인 
train_df.duplicated(['full_log']).sum()

68836

# 레벨별 로그의 특징

In [ ]:
#  
print('level 6 data count: ', len(train_df[train_df['level']==6]))
train_df[train_df['level']==6]

level 6 data count:  8


,id,level,full_log
87783,87783,6,Jan 30 08:23:17 localhost sshd[18415]: Bad protocol version identification '\003' from 78.128.113.18 port 1073
170386,170386,6,Feb 8 16:16:47 localhost kernel: device virbr0-nic entered promiscuous mode
213304,213304,6,Mar 8 15:29:30 localhost kernel: device enp2s0 entered promiscuous mode
221847,221847,6,"Jan 29 11:28:59 localhost useradd[88679]: new group: name=test, GID=1001"
223644,223644,6,Jan 22 15:18:49 localhost sshd[19015]: error: maximum authentication attempts exceeded for root from 192.168.0.197 port 61153 ssh2 [preauth]
361823,361823,6,Feb 2 17:22:58 localhost kernel: device virbr0-nic entered promiscuous mode
406796,406796,6,Feb 4 09:59:33 localhost kernel: device virbr0-nic entered promiscuous mode
429489,429489,6,Feb 4 09:59:43 localhost kernel: device enp2s0 entered promiscuous mode


In [ ]:
train_df.loc[train_df['full_log'].str.contains('promiscuous')]

,id,level,full_log
170386,170386,6,Feb 8 16:16:47 localhost kernel: device virbr0-nic entered promiscuous mode
213304,213304,6,Mar 8 15:29:30 localhost kernel: device enp2s0 entered promiscuous mode
361823,361823,6,Feb 2 17:22:58 localhost kernel: device virbr0-nic entered promiscuous mode
406796,406796,6,Feb 4 09:59:33 localhost kernel: device virbr0-nic entered promiscuous mode
429489,429489,6,Feb 4 09:59:43 localhost kernel: device enp2s0 entered promiscuous mode


In [ ]:
train_df[train_df['level']==4]

,id,level,full_log
64919,64919,4,Nov 29 05:44:21 localhost sshd[6008]: Did not receive identification string from 211.253.243.66 port 57487
85354,85354,4,Nov 29 22:16:32 sv260 sshd[39585]: Did not receive identification string from 211.253.243.66
123069,123069,4,Nov 3 10:38:05 localhost sshd[76373]: Did not receive identification string from 192.168.0.195 port 3819
167164,167164,4,Nov 21 01:02:56 sv260 sshd[9196]: Did not receive identification string from 211.253.243.66
184483,184483,4,Nov 27 08:32:12 sv260 sshd[3110]: Did not receive identification string from 192.168.0.195
230131,230131,4,Nov 19 01:03:09 sv260 sshd[13841]: Did not receive identification string from 192.168.0.195
321526,321526,4,Nov 12 04:27:24 localhost sshd[98418]: Did not receive identification string from 61.41.101.142 port 8975
348055,348055,4,Jan 27 20:20:54 localhost sshd[5025]: Did not receive identification string from 35.184.222.44 port 46232
348684,348684,4,Nov 3 06:03:17 localhost sshd[4346]: Did not receive identification string from 192.168.0.181 port 82066
369842,369842,4,Nov 8 20:25:42 sv260 sshd[37597]: Did not receive identification string from 192.168.0.181


In [ ]:
train_df[train_df['level']==2]

,id,level,full_log
39057,39057,2,The average number of logs between 10:00 and 11:00 is 5399. We reached 37090.
87600,87600,2,The average number of logs between 5:00 and 6:00 is 222. We reached 77946.
109510,109510,2,The average number of logs between 6:00 and 7:00 is 6420. We reached 28494.
262892,262892,2,The average number of logs between 21:00 and 22:00 is 5325. We reached 95.
294072,294072,2,The average number of logs between 5:00 and 6:00 is 8362. We reached 93476.
294127,294127,2,The average number of logs between 4:00 and 5:00 is 2806. We reached 11476.
343011,343011,2,The average number of logs between 14:00 and 15:00 is 9734. We reached 26958.
345903,345903,2,The average number of logs between 15:00 and 16:00 is 4411. We reached 9487.
365348,365348,2,Sep 18 11:02:59 localhost sudo: apache : TTY=unknown ; PWD=/var/www/html/management ; USER=root ; COMMAND=/bin/curl -XGET localhost:9200/_cat/snapshots/esild_backup
376681,376681,2,The average number of logs between 2:00 and 3:00 is 9428. We reached 75572.


In [ ]:
# 레벨별 데이터프레임으로 나눠줌
level0_log = train_df[train_df['level'] == 0]
level1_log = train_df[train_df['level'] == 1]
level2_log = train_df[train_df['level'] == 2]
level3_log = train_df[train_df['level'] == 3]
level4_log = train_df[train_df['level'] == 4]
level5_log = train_df[train_df['level'] == 5]
level6_log = train_df[train_df['level'] == 6]

In [ ]:
# level 5의 첫번째 단어만 출력
level5_words = level5_log['full_log'].str.extract('^([^ ]+)(?: |$)')
level5_words

,0
590,ossec:
621,ossec:
794,File
870,ossec:
1076,level
...,...
471837,ossec:
471864,ossec:
472052,File
472400,ossec:


In [ ]:
level5_words.value_counts()

ossec:      1289
File         641
oscap:       168
level         65
NTFS          25
Trojaned      15
Jan           14
Nov            1
Feb            1
dtype: int64

In [ ]:
len(level5_log[level5_log['full_log'].str.contains('Feb')])

1

In [ ]:
sample_df_log5 = pd.DataFrame(columns=['id', 'level', 'full_log'])

for i in level5_words[0].unique():
    if len(level5_log[level5_log['full_log'].str.contains(i)]) > 3:
        temp_df = level5_log[level5_log['full_log'].str.contains(i)].sample(3)
    else:
        n = len(level5_log[level5_log['full_log'].str.contains(i)])
        temp_df = level5_log[level5_log['full_log'].str.contains(i)].sample(n)
    sample_df_log5 = pd.concat([sample_df_log5, temp_df])


In [ ]:
sample_df_log5

,id,level,full_log
193877,193877,5,ossec: output: 'netstat listening ports':\ntcp 211.253.243.71.49203 211.253.243.71.49182\ntcp 211.253.243.71.49182 211.253.243.71.49203\ntcp 211.253.243.71.49216 211.253.243.71.6000\ntcp 211.253.243.71.6000 211.253.243.71.49188\ntcp 211.253.243.71.6000 211.253.243.71.49187\ntcp 127.0.0.1.49206 127.0.0.1.49205\ntcp 127.0.0.1.49205 127.0.0.1.49206\ntcp 211.253.243.71.49241 211.253.243.71.6000\ntcp 211.253.243.71.49182 211.253.243.71.49236\ntcp 211.253.243.71.6000 211.253.243.71.49181\ntcp 211.253.243.71.49229 211.253.243.71.6000\ntcp 211.253.243.71.49230 211.253.243.71.49182\ntcp 211.253.243.71.57753 61.41.101.142.1514\ntcp 211.253.243.71.6000 211.253.243.71.49202\ntcp 211.253.243.71.6000 211.253.243.71.49201\ntcp 211.253.243.71.49202 211.253.243.71.6000\ntcp 211.253.243.71.6000 211.253.243.71.49200\ntcp 211.253.243.71.49200 211.253.243.71.6000\ntcp 211.253.243.71.49201 211.253.243.71.6000\ntcp 211.253.243.71.22 61.41.101.142.52709\ntcp 211.253.243.71.6000 211.253.243.71.49170\ntcp 211.253.243.71.6000 211.253.243.71.49229\ntcp 211.253.243.71.49182 211.253.243.71.49230\ntcp 211.253.243.71.49182 211.253.243.71.49195\ntcp 127.0.0.1.49198 127.0.0.1.49197\ntcp 127.0.0.1.49197 127.0.0.1.49198\ntcp 127.0.0.1.49226 127.0.0.1.49227\ntcp 211.253.243.71.49235 211.253.243.71.6000\ntcp 211.253.243.71.49194 211.253.243.71.6000\ntcp 211.253.243.71.6000 211.253.243.71.49194\ntcp 211.253.243.71.49195 211.253.243.71.49182\ntcp 127.0.0.1.49227 127.0.0.1.49226\ntcp 127.0.0.1.49233 127.0.0.1.49232\ntcp 211.253.243.71.6000 211.253.243.71.49241\ntcp 211.253.243.71.49170 211.253.243.71.6000\ntcp 127.0.0.1.49232 127.0.0.1.49233\ntcp 211.253.243.71.6000 211.253.243.71.49215\ntcp 211.253.243.71.49215 211.253.243.71.6000\ntcp 127.0.0.1.49238 127.0.0.1.49239\ntcp 211.253.243.71.6000 211.253.243.71.49216\ntcp 127.0.0.1.49239 127.0.0.1.49238\ntcp 211.253.243.71.6000 211.253.243.71.49223\ntcp 211.253.243.71.49224 211.253.243.71.49182\ntcp 211.253.243.71.49182 211.253.243.71.49224\ntcp 211.253.243.71.49181 211.253.243.71.6000\ntcp 211.253.243.71.49236 211.253.243.71.49182\ntcp 211.253.243.71.6000 211.253.243.71.49235\ntcp 211.253.243.71.49223 211.253.243.71.6000\ntcp 211.253.243.71.49188 211.253.243.71.6000\ntcp 211.253.243.71.49187 211.253.243.71.6000\ntcp 211.253.243.71.57990 211.253.243.71.1712\nudp 127.0.0.1.49192 127.0.0.1.49192
41164,41164,5,ossec: output: 'netstat listening ports':\ntcp 211.253.243.71.49391 61.41.101.142.1514\ntcp 211.253.243.71.49177 211.253.243.71.6000\ntcp 211.253.243.71.6000 211.253.243.71.49172\ntcp 211.253.243.71.49172 211.253.243.71.6000\ntcp 211.253.243.71.6000 211.253.243.71.49177\ntcp 211.253.243.71.6000 211.253.243.71.49170\ntcp 211.253.243.71.49170 211.253.243.71.6000\ntcp 211.253.243.71.52969 211.253.243.71.1712\nudp 127.0.0.1.49164 127.0.0.1.49164
374158,374158,5,ossec: output: 'netstat listening ports':\ntcp *.* *.* CLOSED\ntcp *.111 *.* LISTEN\ntcp *.14248 *.* LISTEN\ntcp *.16191 *.* LISTEN\ntcp *.199 *.* LISTEN\ntcp *.21 *.* LISTEN\ntcp *.23 *.* LISTEN\ntcp *.32770 *.* LISTEN\ntcp *.32771 *.* LISTEN\ntcp *.32773 *.* LISTEN\ntcp *.32782 *.* LISTEN\ntcp *.32785 *.* LISTEN\ntcp *.32792 *.* LISTEN\ntcp *.427 *.* LISTEN\ntcp *.512 *.* LISTEN\ntcp *.513 *.* LISTEN\ntcp *.514 *.* LISTEN\ntcp *.5335 *.* LISTEN\ntcp *.5336 *.* LISTEN\ntcp *.5988 *.* LISTEN\ntcp *.5989 *.* LISTEN\ntcp *.6181 *.* LISTEN\ntcp *.657 *.* LISTEN\ntcp *.6987 *.* LISTEN\ntcp *.6988 *.* LISTEN\ntcp *.9510 *.* LISTEN\ntcp4 *.* *.* CLOSED\ntcp4 *.13 *.* LISTEN\ntcp4 *.1334 *.* LISTEN\ntcp4 *.22 *.* LISTEN\ntcp4 *.25 *.* LISTEN\ntcp4 *.32768 *.* LISTEN\ntcp4 *.32769 *.* LISTEN\ntcp4 *.37 *.* LISTEN\ntcp4 *.4000 *.* LISTEN\ntcp4 *.6000 *.* LISTEN\ntcp4 *.6112 *.* LISTEN\ntcp4 127.0.0.1.45957 127.0.0.1.9515 TIME_WAIT\ntcp4 127.0.0.1.9514 *.* LISTEN\ntcp4 127.0.0.1.9515 *.* LISTEN\ntcp4 211.253.243.72.33255 61.41.101.142.1514 ESTABLISHED\ntcp6 *.22 *.* LISTEN\ntcp6 *.6000 *.* LISTEN\ntcp6 ::1.32783 ::1.32784 CLOSE_WAIT\nudp *.

In [ ]:
# level 3의 첫번째 단어만 출력
level3_words = level3_log['full_log'].str.extract('^([^ ]+)(?: |$)')
level3_words

,0
337,Jan
515,File
638,Jan
1058,Jan
1560,Jan
...,...
472741,Jan
472753,Jan
472857,Nov
472926,Jan


In [ ]:
level3_words.value_counts()

Jan        3414
Nov        239 
oscap:     198 
File       181 
Feb        44  
2021       22  
Dec        14  
level      11  
2020       9   
Oct        6   
juniper    1   
Sep        1   
Mar        1   
dtype: int64

In [ ]:
sample_df_log3 = pd.DataFrame(columns=['id', 'level', 'full_log'])

for i in level3_words[0].unique():
    if len(level3_log[level3_log['full_log'].str.contains(i)]) > 3:
        temp_df = level3_log[level3_log['full_log'].str.contains(i)].sample(3)
    else:
        n = len(level3_log[level3_log['full_log'].str.contains(i)])
        temp_df = level3_log[level3_log['full_log'].str.contains(i)].sample(n)
    sample_df_log3 = pd.concat([sample_df_log3, temp_df])


In [ ]:
sample_df_log3

,id,level,full_log
184938,184938,3,Jan 28 22:50:30 localhost sshd[28451]: Failed password for invalid user nginx from 95.111.236.177 port 42368 ssh2
132852,132852,3,Jan 27 20:32:55 localhost sshd[6916]: Invalid user git from 35.184.222.44 port 45004
61555,61555,3,Jan 22 08:06:11 localhost systemd: Unit esild-ml.service entered failed state.
99766,99766,3,File 'HKEY_LOCAL_MACHINE\System\CurrentControlSet\Services\CaptureService_60e249c\Security' was added.\n
245439,245439,3,File 'HKEY_LOCAL_MACHINE\System\CurrentControlSet\Services\CldFlt' was added.\n
422223,422223,3,File 'HKEY_LOCAL_MACHINE\System\CurrentControlSet\Services\WbioSrvc\Service Providers\FacialFeatures\Virtual Sensors\{0527b250-7514-4321-8b68-41c65f956998}' was added.\n
456070,456070,3,"oscap: msg: ""xccdf-result"", scan-id: ""0001602316806"", content: ""ssg-centos-7-ds.xml"", title: ""Record Events that Modify the System's Discretionary Access Controls - setxattr"", id: ""xccdf_org.ssgproject.content_rule_audit_rules_dac_modification_setxattr"", result: ""fail"", severity: ""low"", description: ""At a minimum, the audit system should collect file permission changes for all users and root. If the auditd daemon is configured to use the augenrules program to read audit rules during daemon startup (the default), add the following line to a file with suffix .rules in the directory /etc/audit/rules.d: -a always,exit -F arch=b32 -S setxattr -F auid>=1000 -F auid!=4294967295 -F key=perm_mod If the system is 64 bit then also add the following line: -a always,exit -F arch=b64 -S setxattr -F auid>=1000 -F auid!=4294967295 -F key=perm_mod If the auditd daemon is configured to use the auditctl utility to read audit rules during daemon startup, add the following line to /etc/audit/audit.rules file: -a always,exit -F arch=b32 -S setxattr -F auid>=1000 -F auid!=4294967295 -F key=perm_mod If the system is 64 bit then also add the following line: -a always,exit -F arch=b64 -S setxattr -F auid>=1000 -F auid!=4294967295 -F key=perm_mod"", rationale: ""The changing of file permissions could indicate that a user is attempting to gain access to information that would otherwise be disallowed. Auditing DAC modifications can facilitate the identification of patterns of abuse among both authorized and unauthorized users."" references: ""AC-17(7) (http://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-53r4.pdf), AU-1(b) (http://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-53r4.pdf), AU-2(a) (http://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-53r4.pdf), AU-2(c) (http://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-53r4.pdf), AU-2(d) (http://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-53r4.pdf), AU-12(a) (http://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-53r4.pdf), AU-12(c) (http://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-53r4.pdf), IR-5 (http://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-53r4.pdf), 126 (http://iase.disa.mil/stigs/cci/Pages/index.aspx), 172 (http://iase.disa.mil/stigs/cci/Pages/index.aspx), Req-10.5.5 (https://www.pcisecuritystandards.org/documents/PCI_DSS_v3-1.pdf), 5.2.10 (https://benchmarks.cisecurity.org/tools2/linux/CIS_Red_Hat_Enterprise_Linux_7_Benchmark_v1.1.0.pdf), SRG-OS-000064-GPOS-00033 (http://iase.disa.mil/stigs/srgs/Pages/index.aspx), SRG-OS-000392-GPOS-00172 (http://iase.disa.mil/stigs/srgs/Pages/index.aspx), SRG-OS-000458-GPOS-00203 (http://iase.disa.mil/stigs/srgs/Pages/index.aspx), 5.4.1.1 (https://www.fbi.gov/file-repository/cjis-security-policy-v5_5_20160601-2-1.pdf), 3.1.7 (http://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-171.pdf)"", identifiers: """", oval-id: ""oval:ssg-audit_rules_dac_modification_setxattr:def:1"", benchmark-id: ""xccdf_org.ssgproject.content_benchmark_RHEL-7"", profile-id: ""xccdf_org.ssgproject.content_profile_pci-dss"", profile-title: ""PCI-DSS v3 Control Baseline for Red Hat Enterprise Linux 7""."
434657,43

In [ ]:
# level 1의 첫번째 단어만 출력
level1_words = level1_log['full_log'].str.extract('^([^ ]+)(?: |$)')
level1_words

,0
4,type=SYSCALL
5,type=SYSCALL
10,type=SYSCALL
15,type=SYSCALL
22,type=SYSCALL
...,...
472948,Oct
472951,type=SYSCALL
472953,type=SYSCALL
472955,type=SYSCALL


In [ ]:
level1_words.value_counts()

type=SYSCALL     116496
Jan              3019  
Oct              2904  
Nov              2646  
Feb              2381  
Dec              1498  
System           820   
Mar              804   
Sep              799   
level            736   
type=AVC         369   
ossec:           29    
Windows          13    
type=USER_AVC    3     
dtype: int64

In [ ]:
sample_df_log1 = pd.DataFrame(columns=['id', 'level', 'full_log'])

for i in level1_words[0].unique():
    if len(level1_log[level1_log['full_log'].str.contains(i)]) > 3:
        temp_df = level1_log[level1_log['full_log'].str.contains(i)].sample(3)
    else:
        n = len(level1_log[level1_log['full_log'].str.contains(i)])
        temp_df = level1_log[level1_log['full_log'].str.contains(i)].sample(n)
    sample_df_log1 = pd.concat([sample_df_log1, temp_df])


In [ ]:
sample_df_log1

,id,level,full_log
71446,71446,1,"type=SYSCALL msg=audit(1611889947.326:280774): arch=c000003e syscall=2 success=yes exit=8 a0=7f89fd9cc630 a1=0 a2=0 a3=65 items=1 ppid=1 pid=86518 auid=4294967295 uid=0 gid=980 euid=0 suid=0 fsuid=0 egid=980 sgid=980 fsgid=980 tty=(none) ses=4294967295 comm=""ossec-syscheckd"" exe=""/var/esild/bin/ossec-syscheckd"" subj=system_u:system_r:unconfined_service_t:s0 key=""audit-wazuh-r"" type=CWD msg=audit(1611889947.326:280774): cwd=""/"" type=PATH msg=audit(1611889947.326:280774): item=0 name=""/etc/vconsole.conf"" inode=33554502 dev=fd:00 mode=0100644 ouid=0 ogid=0 rdev=00:00 obj=system_u:object_r:locale_t:s0 objtype=NORMAL cap_fp=0000000000000000 cap_fi=0000000000000000 cap_fe=0 cap_fver=0 type=PROCTITLE msg=audit(1611889947.326:280774): proctitle=""/var/esild/bin/ossec-syscheckd"""
258475,258475,1,"type=SYSCALL msg=audit(1603146237.504:125117): arch=c000003e syscall=2 success=yes exit=3 a0=7f8d054cf5a4 a1=80000 a2=1 a3=7f8d056d54f8 items=1 ppid=18785 pid=18786 auid=4294967295 uid=0 gid=0 euid=0 suid=0 fsuid=0 egid=0 sgid=0 fsgid=0 tty=(none) ses=4294967295 comm=""awk"" exe=""/usr/bin/gawk"" subj=system_u:system_r:ksmtuned_t:s0 key=""audit-wazuh-r"" type=CWD msg=audit(1603146237.504:125117): cwd=""/"" type=PATH msg=audit(1603146237.504:125117): item=0 name=""/etc/ld.so.cache"" inode=69008420 dev=fd:00 mode=0100644 ouid=0 ogid=0 rdev=00:00 obj=unconfined_u:object_r:ld_so_cache_t:s0 objtype=NORMAL type=PROCTITLE msg=audit(1603146237.504:125117): proctitle=61776B002F5E284D656D467265657C427566666572737C436163686564293A2F207B66726565202B3D2024327D3B20454E44207B7072696E7420667265657D002F70726F632F6D656D696E666F"
361381,361381,1,"type=SYSCALL msg=audit(1603186801.640:180212): arch=c000003e syscall=59 success=yes exit=0 a0=5556a0e51016 a1=7ffd90290930 a2=5556a0e52da0 a3=8 items=2 ppid=28086 pid=28088 auid=0 uid=0 gid=0 euid=0 suid=0 fsuid=0 egid=0 sgid=0 fsgid=0 tty=(none) ses=226 comm=""sh"" exe=""/usr/bin/bash"" subj=system_u:system_r:system_cronjob_t:s0-s0:c0.c1023 key=""audit-wazuh-x"" type=EXECVE msg=audit(1603186801.640:180212): argc=3 a0=""/bin/sh"" a1=""-c"" a2=2F7573722F6C696236342F73612F73613120312031 type=CWD msg=audit(1603186801.640:180212): cwd=""/root"" type=PATH msg=audit(1603186801.640:180212): item=0 name=""/bin/sh"" inode=167140 dev=fd:00 mode=0100755 ouid=0 ogid=0 rdev=00:00 obj=system_u:object_r:shell_exec_t:s0 objtype=NORMAL type=PATH msg=audit(1603186801.640:180212): item=1 name=""/lib64/ld-linux-x86-64.so.2"" inode=33774210 dev=fd:00 mode=0100755 ouid=0 ogid=0 rdev=00:00 obj=system_u:object_r:ld_so_t:s0 objtype=NORMAL type=PROCTITLE msg=audit(1603186801.640:180212): proctitle=2F62696E2F7368002D63002F7573722F6C696236342F73612F73613120312031"
379154,379154,1,Dec 15 16:10:45 localhost sudo: pam_unix(sudo:session): session opened for user root by (uid=0)
228469,228469,1,Dec 15 16:10:44 localhost sudo: apache : TTY=unknown ; PWD=/var/www/html/management ; USER=root ; COMMAND=/bin/cat /var/esild/ruleset/rules/0325-opensmtpd_rules.xml
437681,437681,1,Dec 15 16:54:52 localhost sudo: pam_unix(sudo:session): session opened for user root by (uid=0)
253972,253972,1,Nov 26 17:00:01 localhost sudo: pam_unix(sudo:session): session opened for user root by (uid=0)
12916,12916,1,Nov 28 04:00:01 localhost sudo: root : TTY=unknown ; PWD=/root ; USER=root ; COMMAND=/var/esild/bin/ossec-control restart
402461,402461,1,Nov 10 16:00:01 localhost sudo: pam_unix(sudo:session): session opened for user root by (uid=0)
321809,321809,1,Oct 13 18:00:01 localhost sudo: root : TTY=unknown ; PWD=/root ; USER=root ; COMMAND=/bin/ping -c 3 192.168.0.0


In [ ]:
# level 0의 첫번째 단어만 출력
level0_words = level0_log['full_log'].str.extract('^([^ ]+)(?: |$)')
level0_words

,0
0,Sep
1,Feb
2,Jan
3,Jan
6,Jan
...,...
472966,Mar
472967,Feb
472969,Oct
472970,Jan


In [ ]:
level0_words.value_counts()

Jan          200574
Feb          33114 
Dec          24370 
Sep          22392 
Oct          20973 
Nov          18980 
Mar          11368 
level        1519  
error:       459   
E:           297   
--MARK--:    12    
juniper      4     
OpenSCAP     3     
dtype: int64

In [ ]:
sample_df_log0 = pd.DataFrame(columns=['id', 'level', 'full_log'])

for i in level0_words[0].unique():
    if len(level0_log[level0_log['full_log'].str.contains(i)]) > 3:
        temp_df = level0_log[level0_log['full_log'].str.contains(i)].sample(3)
    else:
        n = len(level0_log[level0_log['full_log'].str.contains(i)])
        temp_df = level0_log[level0_log['full_log'].str.contains(i)].sample(n)
    sample_df_log0 = pd.concat([sample_df_log0, temp_df])


In [ ]:
sample_df_log0

,id,level,full_log
391469,391469,0,"Sep 24 03:45:29 localhost kibana: {""type"":""error"",""@timestamp"":""2020-09-23T18:45:29Z"",""tags"":[""warning"",""stats-collection""],""pid"":6458,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
102985,102985,0,"Sep 24 13:45:04 localhost kibana: {""type"":""log"",""@timestamp"":""2020-09-24T04:45:04Z"",""tags"":[""error"",""task_manager""],""pid"":6458,""message"":""Failed to poll for work: Error: No Living connections""}"
122461,122461,0,"Sep 24 01:21:08 localhost kibana: {""type"":""error"",""@timestamp"":""2020-09-23T16:21:08Z"",""tags"":[""warning"",""stats-collection""],""pid"":6458,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"
388769,388769,0,Feb 1 13:04:00 localhost suricata[1442]: [1:2010937:3] ET SCAN Suspicious inbound to mySQL port 3306 [Classification: Potentially Bad Traffic] [Priority: 2] {TCP} 192.168.0.160:46542 -> 192.168.0.42:3306
263830,263830,0,"Feb 8 17:58:19 localhost logstash: [2021-02-08T17:58:19,924][INFO ][logstash.outputs.elasticsearch] retrying failed action with response code: 503 ({""type""=>""unavailable_shards_exception"", ""reason""=>""[m-hash-2020.06][0] primary shard is not active Timeout: [1m], request: [BulkShardRequest [[m-hash-2020.06][0]] containing [38] requests]""})"
298819,298819,0,"Feb 8 18:31:24 localhost logstash: [2021-02-08T18:31:24,286][INFO ][logstash.outputs.elasticsearch] retrying failed action with response code: 503 ({""type""=>""unavailable_shards_exception"", ""reason""=>""[m-hash-2020.06][1] primary shard is not active Timeout: [1m], request: [BulkShardRequest [[m-hash-2020.06][1]] containing [17] requests]""})"
40170,40170,0,Jan 22 23:14:06 localhost logstash: 3098 com.mysql.jdbc.SQLError.createCommunicationsException(com/mysql/jdbc/SQLError.java:990)
74988,74988,0,Jan 26 14:12:06 localhost logstash: { 13534 rufus-scheduler intercepted an error:
108772,108772,0,Jan 19 06:30:06 localhost logstash: { 12350 rufus-scheduler intercepted an error:
42835,42835,0,"Dec 24 14:57:49 localhost kibana: {""type"":""error"",""@timestamp"":""2020-12-24T05:57:49Z"",""tags"":[""warning"",""stats-collection""],""pid"":862,""level"":""error"",""error"":{""message"":""No Living connections"",""name"":""Error"",""stack"":""Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)""},""message"":""No Living connections""}"


In [ ]:
b